<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/UV_Measurement_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier werden die Rohdaten des BTS2048-UV-WP in eine nutzbare CSV-Datei geschrieben

Dokumentenname: UV_Measurement_to_CSV.ipynb




Es werden die OR0-Daten (NasaAmes-Format), die eigentlich für den Versand an das BFS gedacht sind umgewandelt und in eine CSV-Datei geschrieben. Aufgrund der großen Datenmenge geschieht dies für jeden Monat getrennt.

In [1]:
# Import der benötigten Module
import os, sys
import glob
import datetime
import pandas as pd
import numpy as np
import csv
import re
import matplotlib.pyplot as plt

from scipy.io import netcdf
from datetime import timedelta
from datetime import datetime

from google.colab import drive
from google.colab import files

**Monat der Exportiert werden soll:**

In [2]:
month = '22.09'

Zu Beginn muss die Google-Drive eingerichtet werden, in der die Messdaten (OR0-Dateien) gespeichert sind. Danach werden alle verfügbaren Unterordner aufgerufen. So wird geprüft, ob der Mount richtig funktioniert hat.

In [3]:
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive'
# Durchsuche den Google Drive-Pfad
for root, dirs, files in os.walk(drive_path):
    for dir in dirs:
        # Gib den Namen des Unterordners aus
        print(os.path.join(root, dir))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab_Notebooks/CSV_Messdaten
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten
/content/drive/MyDrive/Colab_Notebooks/CSV_UVI
/content/drive/MyDrive/Colab_Notebooks/CSV_Gewichtet
/content/drive/MyDrive/Colab_Notebooks/CSV_SolarRadiation
/content/drive/MyDrive/Colab_Notebooks/netCDF4_Wetterdaten
/content/drive/MyDrive/Colab_Notebooks/CAMS_Vorhersage
/content/drive/MyDrive/Colab_Notebooks/Clean_Data
/content/drive/MyDrive/Colab_Notebooks/plot_daily_UVI
/content/drive/MyDrive/Colab_Notebooks/SOLYS_Messdaten
/content/drive/MyDrive/Colab_Notebooks/CSV_Zwischenspeicher
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/22.07
/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/D

Danach werden die einzelnen OR0-Dateien (NasaAmes Format) geladen und umgewandelt.

Das NasaAmes Format: https://espoarchive.nasa.gov/content/Ames_Format_Specification_v20

Die Messdaten sind unter dem FFI (File Format Index) 2005 gespeichert. Dieser Standard ist durch die Nasa nicht dokumentiert. Deshalb nachfolgend ein eigener Parser, der die Daten in eine nutzbare CSV umwandelt.

**Beschreibung der einzelnen Dictionaries und deren Inhalt:**

- **file_names** = enthält die Dateinamen der einzelnen OR0-Dateien
- **file_content** = Enthält den Inhalt der OR0-Dateien
- **end_line_header** = enthält die Zeile an dem der Header endet

**Dateien einlesen:**

In [4]:
# Pfad zum Ordner mit den Dateien in Google Drive
folder_path = '/content/drive/MyDrive/Colab_Notebooks/NasaAmes_Messdaten/Data/' + month

# OR0-Dateien im Ordner lesen
file_paths = glob.glob(folder_path + '/*.OR0')

# Liste für die Dateinamen erstellen
file_names = []

# Schleife über die Dateien
for file_path in file_paths:

    if os.path.getsize(file_path) > 100 * 1024:
      # Dateiname extrahieren
      file_name = os.path.splitext(os.path.basename(file_path))[0]

      # Datei öffnen und Inhalt lesen
      with open(file_path, 'r') as file:
          file_content = file.read()

      # Variable für die Datei erstellen
      globals()[file_name] = file_content

      file_names.append(file_name)

**Dataframe mit Wellenlängen erstellen:**
- Dieser wird später genutzt um die Spalten des Dataframe zu benennen.

In [5]:
def create_df_Wellenlaenge(start, end, step):
    # Erstelle eine Liste mit den gewünschten Werten
    numbers_list = [round(num, 3) for num in list(np.arange(start, end + step, step))]
    # Erstelle den Dataframe
    df = pd.DataFrame({'Wellenlaenge': numbers_list})

    return df

In [6]:
np_Wellenlaenge = np.round(np.arange(290.0, 420.05, 0.1), decimals = 1)
df_Wellenlaenge = pd.DataFrame({'Wellenlaenge': np_Wellenlaenge})

**String aufteilen in einzelne Zeile schreiben:**

- Zur weitern Verarbeitung müssen alle Elemente als einzelene Strings abgespeichet werden.

In [7]:
file_content = {}  # Dictionary für die Messungen erstellen

for file_name in file_names:
    file_variables = globals()[file_name]
    file_content[file_name] = file_variables

# Auf Variablen zugreifen und String in Zeilen aufteilen
for file_name, variable in file_content.items():
    file_content[file_name] = file_content[file_name].split('\n')

**Header extrahieren:**

In [8]:
# Funktion um den Dateiheader zu extrahieren
def perform_action(file_variables, file_names):
    header_dict = {file_name: "" for file_name in file_names}
    end_line_header_fnc = 0

    for file_name, data in file_variables.items():
        for i, line in enumerate(data):
            header_dict[file_name] += line + "\n"
            if line.strip() == "Pyranometer: readout interval [secs]=5":
                end_line_header_fnc = i
                break

    return header_dict, end_line_header_fnc

In [9]:
file_header, end_line_header = perform_action(file_content, file_names)

**Header aus Datensatz löschen:**

In [10]:
def remove_header(lines_content, end_line):
    lines_WO_header_fnc = {}
    lines_WO_header_fnc = lines_content.copy()

    for key, value in lines_WO_header_fnc.items():
        del value[:end_line+1]

    return lines_WO_header_fnc

In [11]:
lines_WO_header = remove_header(file_content, end_line_header)

**Elemente aufteilen:**

In [12]:
def flatten_and_split(input_list):
    result = []
    for sublist in input_list:
        elements = sublist.split()
        result.extend(elements)
    return result

In [13]:
def process_dict(input_dict):
    processed_dict = {}
    for key, value in input_dict.items():
        processed_value = flatten_and_split(value)
        sublists = []
        sublist = []
        for element in processed_value:
            if element.isdigit() and len(element) == 5:
                if sublist:
                    sublists.append(sublist)
                    sublist = []
            sublist.append(element)
        if sublist:
            sublists.append(sublist)
        processed_dict[key] = sublists
    return processed_dict

In [14]:
processed_dict = process_dict(lines_WO_header)

**Zeitstempel in Datensatz finden:**

In [15]:
# Funktion zum finden eines 5 stelligen Integers (= Zeitstempel)
def find_5_digit_integers(input_list):
    result = []
    for sublist in input_list:
        for element in sublist:
            if isinstance(element, str) and element.isdigit() and len(element) == 5:
                result.append(element)
    return result

In [16]:
# Funktion, die durch das dict iterriert
def find_5_digit_integers_in_dict(input_dict):
    result_dict = {}
    for key, value in input_dict.items():
        result_dict[key] = find_5_digit_integers(value)
    return result_dict

In [17]:
result_dict = find_5_digit_integers_in_dict(processed_dict)

**Dict in Dataframe speichern:**

In [18]:
# Erstelle eine leere Liste, um die Zeilen für den DataFrame aufzunehmen
data_rows = []
df_rows = pd.DataFrame()
df_Messdaten = pd.DataFrame()

# Iteriere durch das verschachtelte Dictionary und erstelle Zeilen für den DataFrame
for key, value in processed_dict.items():
    for sublist in value:
        data_rows.append([key] + sublist)

# Definiere Spaltennamen für den DataFrame
columns = ['Datum'] + [f'Wert{i}' for i in range(1, len(data_rows[0]))]

# Erstelle den Pandas DataFrame
df_rows = pd.DataFrame(data_rows, columns=columns)
df_Messdaten = df_rows.copy()

**Prüfen, ob der Inhalt des DF korrekt ist:**

In [19]:
df_rows

,Datum,Wert1,Wert2,Wert3,Wert4,Wert5,Wert6,Wert7,Wert8,Wert9,...,Wert1309,Wert1310,Wert1311,Wert1312,Wert1313,Wert1314,Wert1315,Wert1316,Wert1317,Wert1318
0,SA220906,16320,97,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.8661E-04,7.7102E-04,7.7257E-04,7.7377E-04,7.7653E-04,7.8976E-04,5.1281E-04,0.0000E+00,0.0000E+00,0.0000E+00
1,SA220906,16440,95,-6.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.1227E-03,1.1170E-03,1.1140E-03,1.0805E-03,1.0680E-03,1.0401E-03,1.0363E-03,1.0408E-03,1.0423E-03,1.0564E-03
2,SA220906,16560,95,-6.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.5345E-03,1.5176E-03,1.4873E-03,1.4577E-03,1.4299E-03,1.4139E-03,1.4246E-03,1.4172E-03,1.4134E-03,1.4313E-03
3,SA220906,16680,80,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,2.0209E-03,1.9990E-03,1.9614E-03,1.9108E-03,1.8568E-03,1.8448E-03,1.8441E-03,1.8296E-03,1.8533E-03,1.8479E-03
4,SA220906,16800,65,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,2.5355E-03,2.5455E-03,2.5154E-03,2.4803E-03,2.4475E-03,2.3887E-03,2.3293E-03,2.3383E-03,2.3774E-03,2.3850E-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8250,SA220902,24960,7,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.2486E-01,3.2447E-01,3.2116E-01,3.1517E-01,3.1002E-01,3.0465E-01,3.0057E-01,2.9902E-01,2.9987E-01,3.0271E-01
8251,SA220902,25080,117,-1.360E+00,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00
8252,SA220902,25200,96,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.6574E-01,3.6722E-01,3.6400E-01,3.5634E-01,3.5076E-01,3.4581E-01,3.4155E-01,3.3819E-01,3.3486E-01,3.3738E-01
8253,SA220902,25320,79,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.7690E-01,3.7651E-01,3.7343E-01,3.6593E-01,3.6017E-01,3.5424E-01,3.4859E-01,3.4508E-01,3.4380E-01,3.4732E-01


**Konvertieren des DF:**

In [20]:
# Bestimme die Indexposition, ab der die neuen Spaltennamen zugewiesen werden sollen
start_index = 18  # Beginne ab der 18ten Spalte um an die Messdaten zu kommen

new_column_names = []

# Extrahiere die neuen Spaltennamen aus dem zweiten DataFrame
new_column_names = df_Wellenlaenge['Wellenlaenge'].tolist()
print(new_column_names)

# Ändere die Spaltennamen des DataFrames ab der angegebenen Indexposition
for i, new_name in enumerate(new_column_names):
    df_Messdaten.columns.values[start_index + i] = new_name

[290.0, 290.1, 290.2, 290.3, 290.4, 290.5, 290.6, 290.7, 290.8, 290.9, 291.0, 291.1, 291.2, 291.3, 291.4, 291.5, 291.6, 291.7, 291.8, 291.9, 292.0, 292.1, 292.2, 292.3, 292.4, 292.5, 292.6, 292.7, 292.8, 292.9, 293.0, 293.1, 293.2, 293.3, 293.4, 293.5, 293.6, 293.7, 293.8, 293.9, 294.0, 294.1, 294.2, 294.3, 294.4, 294.5, 294.6, 294.7, 294.8, 294.9, 295.0, 295.1, 295.2, 295.3, 295.4, 295.5, 295.6, 295.7, 295.8, 295.9, 296.0, 296.1, 296.2, 296.3, 296.4, 296.5, 296.6, 296.7, 296.8, 296.9, 297.0, 297.1, 297.2, 297.3, 297.4, 297.5, 297.6, 297.7, 297.8, 297.9, 298.0, 298.1, 298.2, 298.3, 298.4, 298.5, 298.6, 298.7, 298.8, 298.9, 299.0, 299.1, 299.2, 299.3, 299.4, 299.5, 299.6, 299.7, 299.8, 299.9, 300.0, 300.1, 300.2, 300.3, 300.4, 300.5, 300.6, 300.7, 300.8, 300.9, 301.0, 301.1, 301.2, 301.3, 301.4, 301.5, 301.6, 301.7, 301.8, 301.9, 302.0, 302.1, 302.2, 302.3, 302.4, 302.5, 302.6, 302.7, 302.8, 302.9, 303.0, 303.1, 303.2, 303.3, 303.4, 303.5, 303.6, 303.7, 303.8, 303.9, 304.0, 304.1, 304.2

In [21]:
df_Messdaten

,Datum,Wert1,Wert2,Wert3,Wert4,Wert5,Wert6,Wert7,Wert8,Wert9,...,419.1,419.2,419.3,419.4,419.5,419.6,419.7,419.8,419.9,420.0
0,SA220906,16320,97,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,7.8661E-04,7.7102E-04,7.7257E-04,7.7377E-04,7.7653E-04,7.8976E-04,5.1281E-04,0.0000E+00,0.0000E+00,0.0000E+00
1,SA220906,16440,95,-6.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.1227E-03,1.1170E-03,1.1140E-03,1.0805E-03,1.0680E-03,1.0401E-03,1.0363E-03,1.0408E-03,1.0423E-03,1.0564E-03
2,SA220906,16560,95,-6.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,1.5345E-03,1.5176E-03,1.4873E-03,1.4577E-03,1.4299E-03,1.4139E-03,1.4246E-03,1.4172E-03,1.4134E-03,1.4313E-03
3,SA220906,16680,80,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,2.0209E-03,1.9990E-03,1.9614E-03,1.9108E-03,1.8568E-03,1.8448E-03,1.8441E-03,1.8296E-03,1.8533E-03,1.8479E-03
4,SA220906,16800,65,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,2.5355E-03,2.5455E-03,2.5154E-03,2.4803E-03,2.4475E-03,2.3887E-03,2.3293E-03,2.3383E-03,2.3774E-03,2.3850E-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8250,SA220902,24960,7,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.2486E-01,3.2447E-01,3.2116E-01,3.1517E-01,3.1002E-01,3.0465E-01,3.0057E-01,2.9902E-01,2.9987E-01,3.0271E-01
8251,SA220902,25080,117,-1.360E+00,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00
8252,SA220902,25200,96,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.6574E-01,3.6722E-01,3.6400E-01,3.5634E-01,3.5076E-01,3.4581E-01,3.4155E-01,3.3819E-01,3.3486E-01,3.3738E-01
8253,SA220902,25320,79,4.000E-02,9.999E+9,2.000E+00,0.000E+00,999.9,9.999E+9,9.999E+9,...,3.7690E-01,3.7651E-01,3.7343E-01,3.6593E-01,3.6017E-01,3.5424E-01,3.4859E-01,3.4508E-01,3.4380E-01,3.4732E-01


In [22]:
# Splaten in float umwandeln
for spalte in new_column_names:
    # Konvertiere nur, wenn die Spalte im DataFrame existiert
    if str(spalte) in df_Messdaten.columns:
        df_Messdaten[str(spalte)] = df_Messdaten[str(spalte)].astype(float)

In [23]:
# Funktion zum Konvertieren des Datumsformats
def convert_date_format(date_str):
    date_str = date_str[2:]  # Entferne das "SA"-Präfix
    date_obj = datetime.strptime(date_str, "%y%m%d")
    return date_obj

In [24]:
# Wende die Funktion auf die Spalte an
df_Messdaten['Datum'] = df_Messdaten['Datum'].apply(convert_date_format)

DF muss exportiert und wieder importiert werden, damit die Zeilen ordnungsgemäß gelöscht werden können.

In [25]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [26]:
# Dataframe abspeichern und wieder importieren
folder_path2 = '/content/drive/My Drive/Colab_Notebooks/CSV_Zwischenspeicher/'
save_dataframe_to_drive(df_Messdaten, folder_path2, month)

Der DataFrame wurde als 22.09 in /content/drive/My Drive/Colab_Notebooks/CSV_Zwischenspeicher/ auf Google Drive gespeichert.


In [27]:
df_Messdaten_drop = pd.DataFrame()
df_drop = pd.DataFrame()
df_drop = pd.read_csv(folder_path2 + month)

In [28]:
spalten_zum_loeschen = df_drop.columns[2:18]  # Index 3 bis Index 19
df_Messdaten_drop = df_drop.drop(spalten_zum_loeschen, axis=1).copy()

In [29]:
df_Messdaten_drop

,Datum,Wert1,290.0,290.1,290.2,290.3,290.4,290.5,290.6,290.7,...,419.1,419.2,419.3,419.4,419.5,419.6,419.7,419.8,419.9,420.0
0,2022-09-06,16320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000787,0.000771,0.000773,0.000774,0.000777,0.000790,0.000513,0.000000,0.000000,0.000000
1,2022-09-06,16440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001123,0.001117,0.001114,0.001081,0.001068,0.001040,0.001036,0.001041,0.001042,0.001056
2,2022-09-06,16560,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001535,0.001518,0.001487,0.001458,0.001430,0.001414,0.001425,0.001417,0.001413,0.001431
3,2022-09-06,16680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002021,0.001999,0.001961,0.001911,0.001857,0.001845,0.001844,0.001830,0.001853,0.001848
4,2022-09-06,16800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002535,0.002546,0.002515,0.002480,0.002448,0.002389,0.002329,0.002338,0.002377,0.002385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8250,2022-09-02,24960,0.000782,0.000789,0.000844,0.000946,0.000923,0.000881,0.000913,0.001041,...,0.324860,0.324470,0.321160,0.315170,0.310020,0.304650,0.300570,0.299020,0.299870,0.302710
8251,2022-09-02,25080,0.000873,0.000765,0.000800,0.000866,0.000955,0.000903,0.000874,0.000909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8252,2022-09-02,25200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.365740,0.367220,0.364000,0.356340,0.350760,0.345810,0.341550,0.338190,0.334860,0.337380
8253,2022-09-02,25320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.376900,0.376510,0.373430,0.365930,0.360170,0.354240,0.348590,0.345080,0.343800,0.347320


In [30]:
# print(df_Messdaten.isnull().sum(), df_Messdaten_drop.isnull().sum())

In [31]:
df_Messdaten_drop.columns

Index(['Datum', 'Wert1', '290.0', '290.1', '290.2', '290.3', '290.4', '290.5',
       '290.6', '290.7',
       ...
       '419.1', '419.2', '419.3', '419.4', '419.5', '419.6', '419.7', '419.8',
       '419.9', '420.0'],
      dtype='object', length=1303)

Die Splate Uhrzeit gibt es noch nicht und muss noch eingefügt werden.

# Splaten umbennen und ordnen
spalte_uhrzeit = pd.to_datetime(df_Messdaten_drop['Wert1'], unit='s').dt.time

print(spalte_uhrzeit)

position = 1
df_Messdaten_drop.insert(position, 'Uhrzeit', spalte_uhrzeit)

spaltenname = 'Messzeitpunkt'
df_Messdaten_drop.rename(columns={'Wert1': spaltenname}, inplace=True)

In [32]:
# Datetimespalte Einfügen
df_Messdaten_drop['Datum'] = pd.to_datetime(df_Messdaten_drop['Datum'], format="%Y-%m-%d").dt.date

df_Messdaten_drop['Datetime'] = df_Messdaten_drop.apply(lambda row: datetime.combine(row['Datum'], row['Uhrzeit']), axis=1)

df_Messdaten_drop.insert(0, 'Datetime', df_Messdaten_drop.pop('Datetime'))

KeyError: ignored

In [ ]:
df_Messdaten_drop

In [ ]:
save_dataframe_to_drive(df_Messdaten, '/content/drive/My Drive/Colab_Notebooks/CSV_Messdaten', month)

**Allgemeine Informationen über den Dataframe:**

In [ ]:
def finde_max_werte_fuer_alle_spalten(df):
    # Erstellt ein Dictionary, um die maximalen Werte jeder Spalte zu speichern
    max_werte = {}
    id_werte = {}
    # Iteriere über alle Spalten im DataFrame
    for spalte in df.columns:
        # Ignoriere nicht-numerische Spalten
        if pd.api.types.is_numeric_dtype(df[spalte]):
            max_werte[spalte] = df[spalte].max()      # Maximaler Wert
            id_werte[spalte] = df[spalte].idxmax()    # Spalte mit dem höchsten Wert
    return max_werte, id_werte

In [ ]:
# Funktion um den Maximalen Eintrag einer Spalte zu finden
max_werte, id_werte = finde_max_werte_fuer_alle_spalten(df_Messdaten_drop.iloc[:, 4:1306])

In [ ]:
# Plotten der maximalen Werte nach Wellenlänge
schluessel_max = list(max_werte.keys())
werte_max = list(max_werte.values())
plt.plot(schluessel_max, werte_max)

In [ ]:
schluessel_id = list(id_werte.keys())
werte_id = list(id_werte.values())
plt.plot(schluessel_id, werte_id)

In [ ]:
threshold = 1
selected_rows = df_Messdaten_drop[df_Messdaten_drop.iloc[:, 4:1305].gt(threshold).any(axis=1)]

In [ ]:
selected_rows

In [ ]:
nan_einträge = df_Messdaten[df_Messdaten.isna().any(axis=1)]

In [ ]:
nan_einträge, df_Messdaten

am 01.09 und 02.09 gibt es fehlerhafte Einträge. Diese am besten händisch korregieren.